In [74]:
import json
import glob
import csv
import string

In [75]:
filepath = 'comics2event.csv'

In [76]:
out_put = open(filepath, "w+", newline='')
datafile = csv.writer(out_put)      
header = ['event_id','event_id']
datafile.writerow(header)

19

In [77]:
files =  glob.glob('./comic_json/*.json')

In [81]:
with open(files[0]) as data_file:
    data = json.load(data_file)['data']['results']
files[0] 

'./comic_json/comic29600.json'

In [79]:
data[0]['events']['available'] > data[0]['events']['returned'] 

False

In [82]:
data[0]['id']

12729

In [80]:
for i in data[0]['events']['items']:
    print(i['resourceURI'].split('/')[-1])

In [43]:
i = 0
api = []
not_api = []
for fn in files:
    with open(fn) as data_file:
        data = json.load(data_file)['data']['results']
        for comic in data:
            idd = comic['id']
            avail = comic['creators']['available']
            returned = data[0]['creators']['returned'] 
            if avail > 20:
                api.append(idd)
                continue
            
            not_api.append(idd)
            if(len(comic['creators']['items']) == 0):
                i = i + 1
                
            for char in comic['creators']['items']:
                row = []
                row.append(idd)
                row.append(char['resourceURI'].split('/')[-1])
                datafile.writerow(row)
            
                
        
        


In [44]:
api

[10141,
 22417,
 22418,
 22433,
 21300,
 15518,
 10131,
 15511,
 10129,
 1903,
 1909,
 6935,
 10148,
 10146,
 21625,
 21626,
 20740,
 21618,
 23862,
 10041,
 16210,
 16206,
 16222,
 10083,
 10080,
 18156,
 2997,
 957,
 2577,
 23890,
 1500,
 1404,
 1460,
 1498,
 22609,
 28958,
 28840,
 21450,
 21418,
 49970,
 3171,
 3218,
 1313,
 27095,
 1237,
 57439,
 3965,
 12139,
 2001,
 2006,
 2005,
 10150,
 15572,
 1676,
 1672,
 33018,
 17677,
 17676,
 17686,
 13181,
 10171,
 10170,
 17467,
 16897,
 16877,
 25299,
 25877,
 1216,
 1194,
 12148,
 39287,
 42202,
 32631,
 28844,
 5784,
 5858,
 5844,
 5799,
 5785,
 18656,
 5660,
 5705,
 51707,
 22058,
 21813,
 6156,
 17509,
 23302,
 12614,
 6777,
 10021,
 10020,
 10019,
 10018,
 15343,
 23030,
 2523,
 2592,
 2817,
 2841,
 27450,
 16645,
 46086,
 26104,
 24407,
 26106,
 10179,
 10178,
 4778,
 4802,
 17482,
 17465,
 10180,
 7454,
 1025,
 1073,
 1034,
 1025,
 1073,
 1034,
 1040,
 1074,
 10099,
 10096,
 19821,
 5060,
 5132,
 21813,
 21430,
 21295,
 4095,
 4

In [45]:
len(api)

307

In [46]:
len(not_api)

36452

In [47]:
i

4319

In [48]:
out_put.close()

In [49]:
import requests
import hashlib
import datetime
import string
import csv

In [50]:
public_key = "2990a1aae96d40df9612767f23e2cf01"
private_key = "3952fc6b62f02a30c86ee6ced917550f08f743ec"

In [51]:
def get_TS_HASH( pub_key, priv_key):
    ts = str(datetime.datetime.now()).replace(" ", "").replace(".", "").replace("-", "").replace(":", "")
    m = hashlib.md5()
    h = ts + priv_key + pub_key
    m.update( h.encode('utf-8'))
    h = m.hexdigest()
    return ts, h
    

In [52]:
def make_url(base_url,limit,offset, pub_key, priv_key):
    ts, h = get_TS_HASH( pub_key, priv_key)
    get_url = base_url + '?ts=' + ts + '&limit=' + str(limit) + '&offset=' + str(offset) + "&apikey=" + pub_key + '&hash=' + h
    return get_url

In [53]:
out_put = open(filepath, "a", newline='')
datafile = csv.writer(out_put) 

In [54]:
comic_id = api[:]

In [55]:
count = 0

In [56]:
comic_charID_url = 'http://gateway.marvel.com:80/v1/public/comics/%s/creators'

In [57]:
r = requests.get(make_url(comic_charID_url % str(comic_id[1]), 100, count, public_key, private_key))

In [58]:
data = r.json()['data']
total = data['total']

In [59]:
#idd = comic_id.index(5589)

In [60]:
#idd = idd + 2

In [61]:
#comic_id = comic_id[idd:len(comic_id)]

In [62]:
for i in range(len(comic_id)):
    count = 0
    url = (comic_charID_url % comic_id[i])
    r = requests.get(make_url(url, 100, count, public_key, private_key))
    if(r.status_code != 200):
        print("something wrong!!! error code of %d"% r.status_code)
        break
    data = r.json()['data']
    total = data['total']
    while(1):
        count = count + data['count']
        for char in data['results']:
            row = []
            row.append(comic_id[i])
            row.append(char['id'])
            datafile.writerow(row)
        if(count > total):
            break
        if(r.status_code != 200):
            print("something wrong!!! error code of %d"% r.status_code)
            break
        data = r.json()['data']
        
        
        
        
        
    
    
    

In [63]:
out_put.close()